In [1]:
import os
import pandas
import csv
import matplotlib
import matplotlib.pyplot
import cartopy.crs as ccrs
import bokeh
import ipywidgets
import numpy
import functools

In [2]:
from xbt_dataset import XbtDataset, UNKOWN_MODEL_STR

In [3]:
# TODO: access through a intake catalog
xbt_data_dir = '/data/misc/xbt-data/annual_csv/'

In [4]:
print(f'processing data from directory {xbt_data_dir}.')
data_stats = {}
columns = ['year','num obs ''num cruises', 'unknown brand or type',
           'unknown type', 'num_instruments','num_platforms']


processing data from directory /data/misc/xbt-data/annual_csv/.


In [5]:
@functools.lru_cache
def get_dataset(data_dir, year):
    return XbtDataset(data_dir, (year, year+1))


In [43]:
parent_widget = None
def do_plot(year, instrument, show_unknown):
    figure_name = f'xbt_data_plot'
    xbt_year = get_dataset(xbt_data_dir, year)
    if parent_widget:
        parent_widget.widget.children[1].options = xbt_year.instruments + ['all']
    current_figure = matplotlib.pyplot.figure(figure_name, figsize=(16,16))
    projection1 = ccrs.PlateCarree()
    current_axes = current_figure.add_subplot(111, projection=projection1)
    current_axes.coastlines()
    xbt_subset = xbt_year.filter_obs('labelled', show_unknown)
    if instrument == 'all':
        data_subset_df = xbt_subset.xbt_df
    else:
        data_subset_df = xbt_subset.filter_obs('instrument', instrument).xbt_df
    current_axes.plot(numpy.array( data_subset_df.lon), numpy.array(data_subset_df.lat), 'r+')
    current_axes.set_title(f'XBT {year} instrument={instrument}')
    current_axes.set_xlim((-180.0,180.0))
    current_axes.set_ylim((-90.0,90.0))
    current_axes.set_xticks([-180, -120, -60.0, 0, 60, 120, 180], crs=projection1)
    current_axes.set_yticks([-90, -60, -30, 0, 30, 60, 90], crs=projection1)


In [45]:
instr_list = get_dataset(xbt_data_dir, 1991).instruments + ['all']
parent_widget = ipywidgets.interact(do_plot, year=(1966,2016), instrument=instr_list, show_unknown=['all', 'labelled','unlabelled'])

interactive(children=(IntSlider(value=1991, description='year', max=2016, min=1966), Dropdown(description='ins…

In [36]:
xbt_year = get_dataset(xbt_data_dir, 1989)
xbt_year.filter_obs('labelled','unlabelled').xbt_df

,Unnamed: 0,country,lat,lon,date,institute,platform,cruise_number,depth_profile,max_depth,temperature_profile,instrument,temperature_quality_flag,id,model,manufacturer
1242,1242,UNITED STATES,30.233299,-71.199997,19890114,0,USS HARRY E.YARNELL (CG-17; comm-d 1963; decom...,US108146,[ 0. 305. 914. 1097. 1341. 1646. 2134. 243...,3018.0,[20.83 20.83 20.56 20.56 20. 18.89 18.33 18....,XBT: TYPE UNKNOWN,[0 0 0 0 0 0 0 0 0 0],3847833,TYPE UNKNOWN,None
1274,1274,UNITED STATES,28.900000,-70.483299,19890114,0,USS HARRY E.YARNELL (CG-17; comm-d 1963; decom...,US108146,[ 0. 610. 1097. 1128. 1158. 1341. 1585. 182...,2926.0,[22.78 22.78 22.22 21.67 21.11 20.56 19.44 18....,XBT: TYPE UNKNOWN,[0 0 0 0 0 0 0 0 0 0],3847850,TYPE UNKNOWN,None
1354,1354,UNITED STATES,25.650000,-68.866699,19890115,0,USS HARRY E.YARNELL (CG-17; comm-d 1963; decom...,US108146,[ 0. 732. 914. 1219. 1524. 1829. 2134. 243...,2743.0,[24.33 23.61 23.44 22.67 21.67 20.28 19.72 18....,XBT: TYPE UNKNOWN,[0 0 0 0 0 0 0 0 0],3847884,TYPE UNKNOWN,None
2005,2005,UNITED STATES,37.500000,1.766700,19890120,0,USS MACDONOUGH (DL-8/DLG-8/DDG-39; call sign N...,US108106,[ 0. 229. 247. 411. 610. 853. 1067. 128...,2743.0,[14.94 14.94 15.33 15.5 15.28 14.5 13.78 13....,XBT: TYPE UNKNOWN,[0 0 0 0 0 0 0 0 0 0],3848229,TYPE UNKNOWN,None
2687,2687,UNITED STATES,19.333300,-74.633301,19890126,0,USS HARRY E.YARNELL (CG-17; comm-d 1963; decom...,US108146,[ 0. 244. 427. 732. 1036. 1402. 1829. 201...,2804.0,[25.83 25.83 25. 23.89 22.78 21.67 20. 19....,XBT: TYPE UNKNOWN,[0 0 0 0 0 0 0 0 0 0],3848507,TYPE UNKNOWN,None
8520,8520,UNITED STATES,67.033302,12.733300,19890313,0,USS HAYLER,US108221,[ 0. 457. 792. 1219. 1829. 2865.],2865.0,[5. 5. 5.89 6.67 6.83 6.67],XBT: TYPE UNKNOWN,[0 0 0 0 0 0],3850739,TYPE UNKNOWN,None
11580,11580,UNKNOWN,37.849998,4.817000,19890409,0,0,99125577,[ 0. 25. 45. 65. 90. 120. 150. 17...,2697.0,[15.5 15.28 15. 14.5 14.39 13.39 13. 12....,XBT: TYPE UNKNOWN,[0 0 0 0 0 0 0 0 0 0 0],3851792,TYPE UNKNOWN,None
26357,26357,UNITED STATES,31.900000,-118.733299,19890728,0,WADDELL,US108473,[ 0. 183. 213. 259. 305. 396. 518. 61...,2926.0,[18.89 18.44 17.22 15.56 13.89 12.22 10.56 10....,XBT: TYPE UNKNOWN,[0 0 0 0 0 0 0 0 0 0 0],3856858,TYPE UNKNOWN,None
32513,32513,UNITED STATES,38.400002,-72.449997,19890911,0,USS VALDEZ (DE 1096/FF 1096; active 1974 - 1991),US108555,[ 0. 244. 305. 366. 427. 549. 610. 76...,2743.0,[24.17 23.89 19.44 16.39 13.89 13.89 13.06 13....,XBT: TYPE UNKNOWN,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],3858460,TYPE UNKNOWN,None
32937,32937,UNITED STATES,32.400002,-73.466698,19890915,0,USS VALDEZ (DE 1096/FF 1096; active 1974 - 1991),US108555,[ 0. 122. 244. 366. 610. 853. 1036. 121...,2743.0,[28.61 28.61 28.61 28.33 24.89 23.33 22.28 21....,XBT: TYPE UNKNOWN,[0 0 0 0 0 0 0 0 0 0 0 0 0],3858649,TYPE UNKNOWN,None
